In [1]:
from build_dataset import MRILesionDatasetBuilder, LatentImageProcessor

# Build the dataset for the brain generation model
- Inpaint lesions with simple method in the slices with lesions
- Include SHIFTS and WMH2017 datasets

### VH

In [2]:
data_folder="/home/benet/data"
input_folder="VH"
output_folder="VH-SHIFTS-WMH2017_split"
folders=["train", "test"]
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=0.7
seed = 17844
skip_empty_masks=False
fill_lesion=False

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()


# output_folder="lesion2D_VH"
# fill_lesion=False
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

# output_folder="lesion2D_VH_split"
# fill_lesion=False
# train_split=0.7
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 741, train examples: 507 (68.42%), test examples: 234 (31.58%)
In the hole dataset: Total examples: 741, train examples: 507 (68.42%), test examples: 234 (31.58%)


### SHIFTS

In [3]:
data_folder="/home/benet/data"
input_folder="SHIFTS_preprocessedMNI" ###
output_folder="VH-SHIFTS-WMH2017_split"
folders=["dev_in", "dev_out", "eval_in", "train"] ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=0.7
seed = 17844
skip_empty_masks=False
fill_lesion=False

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()


# output_folder="lesion2D_SHIFTS"
# fill_lesion=False
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 1274, train examples: 871 (68.37%), test examples: 403 (31.63%)
In the hole dataset: Total examples: 2015, train examples: 1378 (68.39%), test examples: 637 (31.61%)


### WMH2017

In [4]:
data_folder="/home/benet/data"
input_folder="WMH2017_preprocessedMNI" ###
output_folder="VH-SHIFTS-WMH2017_split"
folders=None ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=0.7
seed = 17844
skip_empty_masks=False
fill_lesion=False

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()


# output_folder="lesion2D_WMH2017"
# fill_lesion=False
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 780, train examples: 546 (70.00%), test examples: 234 (30.00%)
In the hole dataset: Total examples: 2795, train examples: 1924 (68.84%), test examples: 871 (31.16%)


### LATENTS

In [5]:
from pathlib import Path
import os
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
    print(repo_path)
resolution = 256

# output_dir = "/home/benet/data/lesion2D_VH_split/train/latent"
# input_dir = "/home/benet/data/lesion2D_VH_split/train/flair"
output_dir = "/home/benet/data/VH-SHIFTS-WMH2017_split/train/latent"
input_dir = "/home/benet/data/VH-SHIFTS-WMH2017_split/train/flair"

processor_train = LatentImageProcessor(repo_path, output_dir=output_dir, scale=True, 
                                 resolution=resolution, finetuned_vae=False, input_dir=input_dir)
processor_train.process_images()


# output_dir = "/home/benet/data/lesion2D_VH_split/test/latent"
# input_dir = "/home/benet/data/lesion2D_VH_split/test/flair"
output_dir = "/home/benet/data/VH-SHIFTS-WMH2017_split/test/latent"
input_dir = "/home/benet/data/VH-SHIFTS-WMH2017_split/test/flair"
processor_test = LatentImageProcessor(repo_path, output_dir=output_dir, scale=True,
                                    resolution=resolution, finetuned_vae=False, input_dir=input_dir)
processor_test.process_images()

/home/benet/tfg
Device: cuda
Found 1924 images in the dataset folder
7696 latent images saved in /home/benet/data/VH-SHIFTS-WMH2017_split/train/latent
Device: cuda
Found 871 images in the dataset folder
3484 latent images saved in /home/benet/data/VH-SHIFTS-WMH2017_split/test/latent
